In [1]:
from sympy import *
from sympy.vector import CoordSys3D
import numpy as np

Gaus rule

$$
\iiint_V \nabla f \, \mathrm{d}V= \oint_S f\, \vec{n} \,\mathrm{d}S
$$

Assuming a polygon with an $N$ number of edges rotating around the y-axis we can define a parametrized curve

$$
\oint_S f\, \vec{n} \,\mathrm{d}S = \sum^N_e \int_s \int_t f(\vec{r}_e(s, t))\, \vec{n}_e \left| \frac{\partial \vec{r}_e}{\partial s} \times \frac{\partial \vec{r}_e}{\partial t}\right| \, \mathrm{d}s\mathrm{d}t = \sum^N_e \int_s \int_t f(\vec{r}_e(s, t))\left(\frac{\partial \vec{r}_e}{\partial s} \times \frac{\partial \vec{r}_e}{\partial t}\right) \, \mathrm{d}s\mathrm{d}t
$$

where $s \in [0, 1]$ and $t \in [0, 2 \pi)$.
We can write

$$
\vec{r}_e(s, t) = 
\begin{pmatrix}
\cos(t) \big[v_x(e, 0) + s\big\{v_x(e, 1) - v_x(e, 0)\big\} \big] \\
v_y(e, 0) + s\big\{v_y(e, 1) - v_y(e, 0)\big\}  \\
\sin(t) \big[v_x(e, 0) + s\big\{v_x(e, 1) - v_x(e, 0)\big\} \big]
\end{pmatrix}
=
\begin{pmatrix}
\cos(t) \big[v_x(e, 0) + s \Delta X_e \big] \\
v_y(e, 0) + s \Delta Y_e \\
\sin(t) \big[v_x(e, 0) + s \Delta X_e \big]
\end{pmatrix}
$$

and so we get

$$
\frac{\partial \vec{r}_e}{\partial s} = 
\begin{pmatrix}
\cos(t)\Delta X_e\\
\Delta Y_e  \\
\sin(t)\Delta X_e
\end{pmatrix}
$$

and

$$
\frac{\partial \vec{r}_e}{\partial t} = 
\begin{pmatrix}
-\sin(t) \big[v_x(e, 0) + s\Delta X_e\big] \\
0  \\
\cos(t) \big[v_x(e, 0) + s\Delta X_e\big]
\end{pmatrix}
$$

Here $v_x(e, 1)$ is the $x$th componentn of the vertix $1$ of edge $e$.

In [2]:
N = CoordSys3D('N')
v_x0, v_y0= symbols("v_x0 v_y0", real=True, positive=True)
s, t, L, n = symbols("s t L n", real=True, positive=True)
dx, dy, f0, f1  = symbols("dx dy f_0 f_1", real=True)

In [3]:
dr_ds = cos(t)*dx*N.i + dy*N.j + sin(t)*dx*N.k
dr_dt = -sin(t)*(v_x0 + s*dx)*N.i + 0*N.j + cos(t)*(v_x0 + s*dx)*N.k

In [4]:
dr_ds

(dx*cos(t))*N.i + dy*N.j + (dx*sin(t))*N.k

In [5]:
dr_dt

(-(dx*s + v_x0)*sin(t))*N.i + ((dx*s + v_x0)*cos(t))*N.k

In [6]:
cr = dr_ds.cross(dr_dt).simplify()
cr_dot = cr.dot(cr).simplify()

In [18]:
cr

(dy*(dx*s + v_x0)*cos(t))*N.i + (-dx*(dx*s + v_x0))*N.j + (dy*(dx*s + v_x0)*sin(t))*N.k

In [19]:
print(cr)

(dy*(dx*s + v_x0)*cos(t))*N.i + (-dx*(dx*s + v_x0))*N.j + (dy*(dx*s + v_x0)*sin(t))*N.k


In [7]:
cr_dot

(dx**2 + dy**2)*(dx*s + v_x0)**2

In [8]:
cr_dot.subs((dx**2 + dy**2), L**2)

L**2*(dx*s + v_x0)**2

In [9]:
cr_dot = cr_dot.subs((dx**2 + dy**2), L**2)

In [10]:
cr_norm = sqrt(cr_dot)

In [11]:
cr_norm

L*Abs(dx*s + v_x0)

In [12]:
integrate(integrate(cr_norm, (s, 0, 1)), (t, 0, 2*pi)).simplify()

Piecewise((pi*L*(dx + 2*v_x0), dx*s + v_x0 >= 0), (-pi*L*(dx + 2*v_x0), True))

We can write

$$
f(\vec{r}_e(s, t)) = f(e, 0) + s \cdot (f(e, 1) - f(e, 0))
$$

and thus

$$
\vec{n}_e L_e \oint \big[ f(e, 0) + s \cdot (f(e, 1) - f(e, 0)) \big]\, \big| s \cdot \Delta X_e + v(e, 0) \big| \mathrm{d}s \mathrm{d}t
$$

Since $s \cdot \Delta X_e + v_x(e, 0)$ is alway $>0$ we ca write



We need to have

$$
\sum^N_e \int_0^1 \int_0^{2\pi} f(\vec{r}_e(s, t))\left(\frac{\partial \vec{r}_e}{\partial s} \times \frac{\partial \vec{r}_e}{\partial t}\right) \, \mathrm{d}t\mathrm{d}s = \sum^N_e \int_0^1 \int_0^{2\pi}
\bigg[ f(e, 0) + s \cdot (f(e, 1) - f(e, 0)) \bigg]
\begin{pmatrix}
\Delta Y_e (\Delta X_e \cdot s + v_x(e, 0))\cos(t) \\
-\Delta X_e(\Delta X_e \cdot s + v_x(e, 0)) \\
\Delta Y_e(\Delta X_e \cdot s + v_x(e, 0))\sin(t)
\end{pmatrix}
\mathrm{d}t\mathrm{d}s
$$

In [20]:
expr = cr*(f0 + s*(f1 - f0))

In [26]:
print(expr)

(dy*(f_0 + s*(-f_0 + f_1))*(dx*s + v_x0)*cos(t))*N.i + (-dx*(f_0 + s*(-f_0 + f_1))*(dx*s + v_x0))*N.j + (dy*(f_0 + s*(-f_0 + f_1))*(dx*s + v_x0)*sin(t))*N.k


In [32]:
x = (dy*(f0 + s*(-f0 + f1))*(dx*s + v_x0)*cos(t))
y = (-dx*(f0 + s*(-f0 + f1))*(dx*s + v_x0))
z = (dy*(f0 + s*(-f0 + f1))*(dx*s + v_x0)*sin(t))

In [41]:
int_x = integrate(integrate(x, (s, 0, 1)), (t,0, 2*pi))
int_y = integrate(integrate(y, (s, 0, 1)), (t,0, 2*pi)).simplify()
int_z = integrate(integrate(z, (s, 0, 1)), (t,0, 2*pi))

In [43]:
int_x

0

In [44]:
int_y

-pi*dx*(dx*f_0 + 2*dx*f_1 + 3*f_0*v_x0 + 3*f_1*v_x0)/3

In [45]:
int_z

0